# PPO com Stable Baselines3

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap09/cap09-2-PPO-stablebaselines.ipynb)

Vamos usar o algoritmo **PPO** (*Proximal Policy Optimization*) neste Google Colab.

Referências:
- Stable-Baselines3: https://github.com/DLR-RM/stable-baselines3
- Documentação: https://stable-baselines3.readthedocs.io/en/master/index.html


## 1 - Configurações necessárias

### 1.1 Instalação de pacotes

In [40]:
import sys
from IPython.display import clear_output

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
    !pip install "stable-baselines3[extra]==2.0.0"
    !pip install swig
    !pip install gymnasium[box2d]   # works with gymnasium version 0.28.1
    !pip install gymnasium[atari,accept-rom-license]
    #clear_output()

In [41]:
!mkdir log_dir

J� existe uma subpasta ou um arquivo log_dir.


### 1.2 Para salvar vídeo

*Atenção*: Reinicie o ambiente de execução antes de continuar.

In [42]:
if IN_COLAB:
    # Set up fake display; otherwise rendering will fail
    import os
    os.system("Xvfb :1 -screen 0 1024x768x24 &")
    os.environ['DISPLAY'] = ':1'

A gravação é feita com o wrapper [VecVideoRecorder](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder).

In [43]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
  # Start the video at step=0 and record the given number of steps
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  # Close the video recorder
  eval_env.close()

In [44]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

### 1.3 Imports

In [45]:
import gymnasium as gym
import numpy as np
import tensorboard

%load_ext tensorboard

import stable_baselines3
stable_baselines3.__version__

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


'2.1.0'

Abaixo, importamos a classe que representa o algoritmo **PPO** e diversas funções auxiliares.

In [46]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# para treinar com ambientes Atari
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

Abaixo, importamos a classe que vai representar a **rede neural da política**. No caso, importamos uma rede com camadas totalmente conectadas (MLP) e uma rede convolucional (CNN), com arquiteturas (camadas, etc) pré-definidas.

In [47]:
from stable_baselines3.ppo import MlpPolicy, CnnPolicy

## 2 - Cria o ambiente e instancia o agente

Estamos criando o ambiente e passando para a classe **PPO**, que permite testar ou treinar o agente.

Abaixo, criamos o PPO com os valores default dos seus parâmetros mais importantes. Definimos apenas estes:
- `MlpPolicy`, para usar uma MLP com arquitetura pré-definida; para personalizar a arquitetura [veja aqui](https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html)
- `tensorboard_log`, que indica o diretório dos logs
- `verbose=0` para evitar mostrar muitas mensagens.
- para mais informações, veja [a documentação](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html)



In [48]:
# escolha um ID de ambiente do gymnasium
ENV_NAME = "LunarLander-v2"
#ENV_NAME = "BipedalWalker-v3"

env = gym.make(ENV_NAME)

model = PPO(MlpPolicy, env, tensorboard_log="log_dir", verbose=0)

DependencyNotInstalled: Box2D is not installed, run `pip install gymnasium[box2d]`

Para rodar com **jogos de Atari**, descomente o código abaixo.

*Observação*: Talvez você não consiga usar a função para gerar os vídeos.

In [49]:
#'''
# escolha um ID de ambiente correspondente a um game de Atari
# veja em https://gymnasium.farama.org/environments/atari/
ENV_NAME = 'PongNoFrameskip-v4' #'ALE/Pong-v5' 

# Esta função cria o ambiente, junto com wrappers especializados
env = make_atari_env(ENV_NAME, n_envs=4, seed=0)

# Wrapper para fazer cada estado ser uma "pilha" de 4 frames
env = VecFrameStack(env, n_stack=4)

# Cria com uma rede convolucional como política
model = PPO(CnnPolicy, env, tensorboard_log="log_dir", verbose=1)
#'''

NameNotFound: Environment `PongNoFrameskip` doesn't exist.

Vamos avaliar o agente não-treinado:

In [ ]:
# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=30)

print(f"Retorno médio: {mean_reward:.2f} +/- {std_reward:.2f}")

E agora vamos gravar um vídeo para vê-lo em ação, antes de treinado.

In [ ]:
record_video(ENV_NAME, model, video_length=1_000, prefix='ppo-sem-treino')
show_videos('videos', prefix='ppo-sem-treino')

## 3 - Ativa Tensorboard

O comando abaixo abre o **Tensorboard**. Depois, execute o treinamento e acompanhe aqui os resultados, em tempo real.

In [ ]:
%tensorboard --logdir log_dir

## 4 - Treina e salva o agente

Abaixo, rodamos a função de treinamento.

In [ ]:
# Treina por 200 mil passos -- pode demorar mais de 5 min!
model.learn(total_timesteps=200_000)

Depois, gravamos o vídeo do agente treinado.

In [ ]:
record_video(ENV_NAME, model, video_length=1_500, prefix='ppo-treinado')
show_videos('videos', prefix='ppo-treinado')

Por fim, salvamos o agente treinado em arquivo.

In [ ]:
model.save("ppo_model")
del model  # delete trained model from memory to demonstrate loading


## 5 - Carrega e avalia o agente

In [ ]:
model = PPO.load("ppo_model")

In [ ]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=30)

print(f"Retorno médio: {mean_reward:.2f} +/- {std_reward:.2f}")

## 6 - Treinar por mais alguns passos

Para treinar por mais alguns passos, use `.set_env()` para definir o ambiente e chame novamente `.learn()`.

In [ ]:
model.set_env(gym.make(ENV_NAME))
model.learn(total_timesteps=100_000)